In [3]:
import pandas as pd
import numpy as np

In [10]:
pageviews = 5000

In [23]:
df_basevals = pd.read_csv("Final Project Baseline Values.csv", index_col = False, header = None, names = ['metric', 'BaselineValues'])
df_basevals.metric = df_basevals.metric.map(lambda x: x.lower())
df_basevals

,metric,BaselineValues
0,unique cookies to view page per day:,40000.000000
1,"unique cookies to click ""start free trial"" per...",3200.000000
2,enrollments per day:,660.000000
3,"click-through-probability on ""start free trial"":",0.080000
4,"probability of enrolling, given click:",0.206250
5,"probability of payment, given enroll:",0.530000
6,"probability of payment, given click",0.109313


Standard Deviation of Gross Conversion:

In [11]:
round(np.sqrt((.206250*(1-.206250))/(5000*3200/40000)),4)

0.0202

Standard Deviation of Retention:

In [16]:
round(np.sqrt((0.530000*(1-0.530000))/(5000*660/40000)),4)

0.0551

Standard Deviation of Net Conversion:

In [17]:
round(np.sqrt((0.109313*(1-0.109313))/(5000*3200/40000)),4)

0.0156

http://www.evanmiller.org/ab-testing/sample-size.html

# Pageviews without Bonferroni Correction

<b>Gross Conversion</b>

Baseline Conversion Rate: 20.625%

Minimum Detectable Effect: 1%

α: 5%

β: 20%

1 - β: 80%

Sample Size = 25,835 (Enrollments per Group)

Number of Groups = 2 (Experiment and Control)

Total Sample Size = 51,670 enrollments

Clicks/Pageview: 3200/40000 = 0.08 Clicks/Pageview

Pageviews = 645,875


<b>Retention</b>

Baseline Conversion Rate: 53%

Minimum Detectable Effect: 1%

α: 5%

β: 20%

1 - β: 80%

Sample Size = 39,115 (Enrollments per Group)

Number of Groups = 2 (Experiment and Control)

Total Sample Size = 78,230 enrollments

Clicks/Pageview: 660/40000 =  0.0165 Clicks/Pageview

Pageviews = 4,741,212

<b>Net Conversion</b>

Baseline Conversion Rate: 10.9313%

Minimum Detectable Effect: 0.75%

α: 5%

β: 20%

1 - β: 80%

Sample Size = 27,413 (Enrollments per Group)

Number of Groups = 2 (Experiment and Control)

Total Sample Size = 54,826 enrollments

Clicks/Pageview: 3200/40000 = 0.08 Clicks/Pageview

Pageviews = 685,325

# Duration and Exposure

Number of Pageview: 685,325

Fraction of Traffic Exposed:

In [20]:
685325.0/40000

17.133125

Length of Experiment: 18 days

In [25]:
df_control = pd.read_csv("Final Project Results - Control.csv")
df_experiment = pd.read_csv("Final Project Results - Experiment.csv")

In [44]:
results = {"Control":pd.Series([df_control.Pageviews.sum(),df_control.Clicks.sum(),
                                  df_control.Enrollments.sum(),df_control.Payments.sum()],
                                  index = ["Cookies","Clicks","Enrollments","Payments"]),
           "Experiment":pd.Series([df_experiment.Pageviews.sum(),df_experiment.Clicks.sum(),
                               df_experiment.Enrollments.sum(),df_experiment.Payments.sum()],
                               index = ["Cookies","Clicks","Enrollments","Payments"])}
df_results = pd.DataFrame(results)
df_results

,Control,Experiment
Cookies,345543.0,344660.0
Clicks,28378.0,28325.0
Enrollments,3785.0,3423.0
Payments,2033.0,1945.0


# Sanity Checks

In [45]:
df_results['Total'] = df_results.Control + df_results.Experiment
df_results['Prob'] = 0.5
df_results['StdErr'] = np.sqrt((df_results.Prob * (1-df_results.Prob))/df_results.Total)
df_results['MargErr'] = 1.96 * df_results.StdErr
df_results['CI_lower'] = df_results.Prob - df_results.MargErr
df_results['CI_upper'] = df_results.Prob + df_results.MargErr
df_results['Obs_val'] = df_results.Experiment/df_results.Total
df_results['Pass_Sanity'] = df_results.apply(lambda x: (x.Obs_val > x.CI_lower) and (x.Obs_val < x.CI_upper), axis=1)
df_results['Diff'] = abs((df_results.Experiment - df_results.Control)/df_results.Total)

df_results

,Control,Experiment,Total,Prob,StdErr,MargErr,CI_lower,CI_upper,Obs_val,Pass_Sanity,Diff
Cookies,345543.0,344660.0,690203.0,0.5,0.000602,0.001180,0.498820,0.501180,0.499360,True,0.001279
Clicks,28378.0,28325.0,56703.0,0.5,0.002100,0.004116,0.495884,0.504116,0.499533,True,0.000935
Enrollments,3785.0,3423.0,7208.0,0.5,0.005889,0.011543,0.488457,0.511543,0.474889,False,0.050222
Payments,2033.0,1945.0,3978.0,0.5,0.007928,0.015538,0.484462,0.515538,0.488939,True,0.022122
